# ArangoDB QA chain

This notebook shows how to use LLMs to provide a natural language interface to an [ArangoDB](https://github.com/arangodb/arangodb#readme) database.

You can get a local ArangoDB instance running via the [ArangoDB Docker image](https://hub.docker.com/_/arangodb):  

```
docker run -p 8529:8529 -e ARANGO_ROOT_PASSWORD= arangodb/arangodb
```

An alternative is to use the [ArangoDB Cloud Connector package](https://github.com/arangodb/adb-cloud-connector#readme) to get a temporary cloud instance running.

In [14]:
# Let's install the ArangoDB Python Driver
%%capture
!pip install python-arango

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ArangoDBGraphQAChain
from langchain.graphs import ArangoDBGraph

In [5]:
from arango import ArangoClient

# Instantiate the ArangoDB Python Client
db = ArangoClient(hosts="localhost:8529").db(
    name="_system", username="root", password="", verify=True
)

# Instantiate the ArangoDB-LangChain Graph
graph = ArangoDBGraph(db)

## Populating the Database

We will rely on the Python Driver to import our [GameOfThrones](https://github.com/arangodb/example-datasets/tree/master/GameOfThrones) data into our database.

In [6]:
if db.has_graph("GameOfThrones"):
    db.delete_graph("GameOfThrones", drop_collections=True)

db.create_graph(
    "GameOfThrones",
    edge_definitions=[
        {
            "edge_collection": "ChildOf",
            "from_vertex_collections": ["Characters"],
            "to_vertex_collections": ["Characters"],
        },
    ],
)

documents = [
    {
        "_key": "NedStark",
        "name": "Ned",
        "surname": "Stark",
        "alive": True,
        "age": 41,
        "gender": "male",
    },
    {
        "_key": "CatelynStark",
        "name": "Catelyn",
        "surname": "Stark",
        "alive": False,
        "age": 40,
        "gender": "female",
    },
    {
        "_key": "AryaStark",
        "name": "Arya",
        "surname": "Stark",
        "alive": True,
        "age": 11,
        "gender": "female",
    },
    {
        "_key": "BranStark",
        "name": "Bran",
        "surname": "Stark",
        "alive": True,
        "age": 10,
        "gender": "male",
    },
]

edges = [
    {"_to": "Characters/NedStark", "_from": "Characters/AryaStark"},
    {"_to": "Characters/NedStark", "_from": "Characters/BranStark"},
    {"_to": "Characters/CatelynStark", "_from": "Characters/AryaStark"},
    {"_to": "Characters/CatelynStark", "_from": "Characters/BranStark"},
]

db.collection("Characters").import_bulk(documents)
db.collection("ChildOf").import_bulk(edges)

{'error': False,
 'created': 4,
 'errors': 0,
 'empty': 0,
 'updated': 0,
 'ignored': 0,
 'details': []}

## Getting & Setting the ArangoDB Schema

An initial ArangoDB Schema is generated upon instantiating the `ArangoDBGraph` object. Below are the schema's getter & setter methods should you be interested in viewing or modifying the schema:

In [8]:
# The schema should be empty here,
# since `graph` was initialized prior to ArangoDB Data ingestion (see above).

import json

print(json.dumps(graph.schema, indent=4))

{
    "Graph Schema": [],
    "Collection Schema": []
}


In [9]:
graph.set_schema()

In [10]:
# We can now view the generated schema

import json

print(json.dumps(graph.schema, indent=4))

{
    "Graph Schema": [
        {
            "graph_name": "GameOfThrones",
            "edge_definitions": [
                {
                    "edge_collection": "ChildOf",
                    "from_vertex_collections": [
                        "Characters"
                    ],
                    "to_vertex_collections": [
                        "Characters"
                    ]
                }
            ]
        }
    ],
    "Collection Schema": [
        {
            "collection_name": "ChildOf",
            "collection_type": "edge",
            "edge_properties": {
                "_key": "str",
                "_id": "str",
                "_from": "str",
                "_to": "str"
            },
            "example_edge": {
                "_key": "266215846076",
                "_id": "ChildOf/266215846076",
                "_from": "Characters/AryaStark",
                "_to": "Characters/NedStark",
                "_rev": "_gUV5dAm---"
            }
     

## Querying the ArangoDB Database

We can now use the ArangoDB Graph QA Chain to inquire about our data

In [15]:
chain = ArangoDBGraphQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [16]:
chain.run("Is Ned Stark alive?")



> Entering new ArangoDBGraphQAChain chain...
Generated AQL:

FOR character IN Characters
FILTER LOWER(character.name) == "ned" AND LOWER(character.surname) == "stark"
RETURN character.alive

AQL Result:
[True]

> Finished chain.


'Yes, Ned Stark is alive.'

In [17]:
chain.run("How old is Arya Stark?")



> Entering new ArangoDBGraphQAChain chain...
Generated AQL:

FOR character IN Characters
FILTER character.name == "Arya" AND character.surname == "Stark"
RETURN character.age

AQL Result:
[11]

> Finished chain.


'Arya Stark is 11 years old.'

In [22]:
chain.run("Who is the child of Ned Stark?")



> Entering new ArangoDBGraphQAChain chain...
Generated AQL:

FOR child IN ChildOf
    FILTER child._to == 'Characters/NedStark'
    RETURN child

AQL Result:
[{'_key': '266215846076', '_id': 'ChildOf/266215846076', '_from': 'Characters/AryaStark', '_to': 'Characters/NedStark', '_rev': '_gUV5dAm---'}, {'_key': '266215846077', '_id': 'ChildOf/266215846077', '_from': 'Characters/BranStark', '_to': 'Characters/NedStark', '_rev': '_gUV5dAm--_'}]

> Finished chain.


'The child of Ned Stark is Arya Stark and Bran Stark.'